In [1]:
import json
from pprint import pprint

In [2]:
with open("Data/JSON/PCS_Index.json") as index_file:
    index = json.load(index_file)
with open("Data/JSON/PCS_Tables.json") as table_file:
    tables = json.load(table_file)
with open("Data/JSON/PCS_Definitions.json") as definitions_file:
    defs = json.load(definitions_file)

In [3]:
class Parser:
    def execute_tree(self, mainterm, single_level_check=False):
        """
        Args:
            Mainterm -> Mainterm in focus.
            single_level_check -> Bool value signaling to check for single level.
        Returns:
            None.
        """
        levels = self.check_for_levels(mainterm)
        if levels == "level_1" and single_level_check == True:
            print("--------EXECUTE SINGLE LEVEL--------")
            SingleLevel().execute_single_level(mainterm)
        elif not levels:
            print("--------PARENT EXECUTE--------")
            ParentExecute().parent_execute(mainterm)
        else:
            print("--------PROGRESS THROUGH LEVELS--------")
            self.progress_through_levels(mainterm)
    
    def progress_through_levels(self, mainterm):
        """
        Args
            mainterm -> Mainterm in focus.
        Returns:
            None.
        """
        level_flag = True
        while level_flag:
            new_level_terms = self.get_next_level_choices(mainterm)
            if new_level_terms == None:
                ParentExecute().execute_group_3(mainterm)
                break
            elif len(new_level_terms) == 1:
                print(f"Automatically choosing term, '{new_level_terms[0]}', because it's the only choice available.")
                mainterm = [i for i in mainterm["term"] if i["title"] == new_level_terms[0]][0]
            else:
                print(new_level_terms)
                user_input = input("Choose term : ")
                mainterm = [i for i in mainterm["term"] if i["title"] == user_input][0]
                levels = self.check_for_levels(mainterm)
                if not levels:
                    print("--------EXECUTE_GROUP_3--------")
                    ParentExecute().execute_group_3(mainterm)
                    level_flag = False
                
    def check_for_levels(self, mainterm):
        """
        Args
            mainterm -> mainterm in focus.
        Returns
            level_1 -> Indicates multiple terms on level 1 to be handled, no further levels.
            True -> Indicates multiple levels to be handled.
            False -> Indicates no levels to be handled.
        """
        try:
            for sub_term in mainterm["term"]:
                try:
                    if sub_term["term"]:
                        return True
                except KeyError as err:
                    pass
            try:
                if mainterm["term"][0]["_level"]:
                    return "level_1"
            except KeyError as err:
                return False
        except KeyError as err:
            return False
        
    def get_next_level_choices(self, mainterm):
        """
        Args
            mainterm -> Mainterm in focus.
        Returns
            List of "title" values found on next level of mainterm.
            If no next level, returns None.
        """
        try:
            next_level_terms = [level["title"] for level in mainterm["term"]]
        except KeyError as err:
            next_level_terms = None
        return next_level_terms
    
    def check_for_same_structure(self, mainterm, set_like_object):
        """
        Args
            mainterm -> Mainterm in focus.
            set_like_object -> set containing ordered dictionary keys which are being compared.
        Returns
            None
        """
        for term in mainterm["term"]:
            if set_like_object != term.keys():
                keys = []
                for i in mainterm["term"]:
                    if i.keys() not in keys:
                        keys.append(i.keys())
                # ADD FUNCTIONALITY FOR VARYING SUBTERM STRUCTURES
                if [{'title', 'codes', '_level'}, {'title', 'code', '_level'}] == keys:
                    pass
                elif [{'title', 'code', '_level'}, {'title', 'codes', '_level'}] == keys:
                    pass
                else:
                    raise ValueError("The subterm's of the mainterm being iterated have a varying 'key:value' structure.")
    
    def render_next_level_choices(self, mainterm, key_1, key_2=None):
        """
        Args
            mainterm -> Mainterm in focus.
            key_1 -> String representing a dictionary key.
            key_2 -> String representing a dictionary key.
        Return
            User's choice of next level term.
        """
        if key_2:
            text_choices = [term[key_1][key_2] for term in mainterm["term"]]
        else:
            text_choices = [term[key_1] for term in mainterm["term"]]
        print(text_choices)
        return input("Choose term that most correlates to this medical case: ")

In [4]:
class SingleLevel(Parser):
    def execute_single_level(self, mainterm):
        """
        Args
            mainterm -> Mainterm in focus.
        Returns
            None.
        """
        # First subterm used to understand mainterm's structure
        # ADD FUNCTIONALITY FOR VARYING SUBTERM STRUCTURES
        first_subterm = mainterm["term"][0]
        if {"see", "_level"} == first_subterm.keys():
            self.check_for_same_structure(mainterm, {"see", "_level"})
            self.execute_single_level_1_1(mainterm)
        if {"title", "see", "_level"} == first_subterm.keys():
            self.check_for_same_structure(mainterm, {"title", "see", "_level"})
            self.execute_single_level_3(mainterm)    
        if {"use", "_level"} == first_subterm.keys():
            self.check_for_same_structure(mainterm, {"use", "_level"})
            self.execute_single_level_4(mainterm)
        if {"code", "_level"} == first_subterm.keys():
            self.check_for_same_structure(mainterm, {"code", "_level"})
            self.execute_single_level_5(mainterm)
        if {'title', 'code', '_level'} == first_subterm.keys():
            self.check_for_same_structure(mainterm, {'title', 'code', '_level'})
            self.execute_tree(new_mainterm)
        if {'title', 'codes', '_level'} == first_subterm.keys():
            self.check_for_same_structure(mainterm, {"title", "see", "_level"})
            self.execute_single_level_7(mainterm)

    def execute_single_level_1(self, mainterm):
        print("--------execute_single_level_1--------")
        """
        Args
            mainterm -> Mainterm in focus.
        Returns
            None.
        """
        user_input = self.render_next_level_choices(mainterm, "see")
        for new_mainterm in mainterms: # mainterms -> list of mainterms, global variable must be established
            if new_mainterm["title"] == user_input:
                self.execute_tree(new_mainterm)
                break
    
    def execute_single_level_1_1(self, mainterm):
        print("--------execute_single_level_1_1--------")
        """
        Args
            mainterm -> Mainterm in focus.
        Returns
            None.
        """
        first_subterm = mainterm["term"][0]
        # ADD FUNCTIONALITY FOR VARYING SUBTERM STRUCTURES
        if isinstance(first_subterm["see"], str):
            self.execute_single_level_1(mainterm)   
        elif {"tab", "__text"} == first_subterm["see"].keys():
            self.execute_single_level_2(mainterm)
        elif {"codes", "__text"} == first_subterm["see"].keys():
            self.execute_single_level_2(mainterm)
    
    def execute_single_level_2(self, mainterm):
        print("--------execute_single_level_2--------")
        """
        Args
            mainterm -> Mainterm in focus.
        Returns
            None.
        """
        user_input = self.render_next_level_choices(mainterm, "see", "__text")
        for new_mainterm in mainterm["term"]:
            if new_mainterm["see"]["__text"] == user_input:
                self.execute_tree(new_mainterm)
                break
    
    def execute_single_level_3(self, mainterm):
        print("--------execute_single_level_3--------")
        """
        Args
            mainterm -> Mainterm in focus.
        Returns
            None
        """
        user_input = self.render_next_level_choices(mainterm, "title")
        for new_mainterm in mainterm["term"]:
            if new_mainterm["title"] == user_input:
                self.execute_tree(new_mainterm)
                break
                
    def execute_single_level_4(self, mainterm):
        print("--------execute_single_level_4--------")
        """
        Args
            mainterm -> Mainterm in focus.
        Returns
            None
        """
        # ADD FUNCTIONALITY FOR VARYING SUBTERM STRUCTURES
        first_subterm = mainterm["term"][0]
        if isinstance(first_subterm["use"], str):
            user_input = self.render_next_level_choices(mainterm, "use")
            for new_mainterm in mainterm["term"]:
                if new_mainterm["use"] == user_input:
                    self.execute_tree(new_mainterm)
                    break
        else:
            if {"tab", "__text"} == first_subterm["use"].keys():
                user_input = self.render_next_level_choices(mainterm, "use", "__text")
                for new_mainterm in mainterm["term"]:
                    if new_mainterm["use"]["__text"] == user_input:
                        self.execute_tree(new_mainterm)
                        break 
            else:
                print("ERROR: UN-ACCOUNTED FOR SUBTERM KEYS -> execute_single_level_4()")
                
    def execute_single_level_5(self, mainterm):
        print("--------execute_single_level_5--------")
        """
        Args
            mainterm -> Mainterm in focus.
        Returns
            None
        """
        len_choices = len(mainterm["term"])
        choices = [i["code"] for i in mainterm["term"]]
        print(f"You have {len_choices} choices for final codes related to term {mainterm['title']} Choose the code that works best for this medical case.")
        print(choices)
        
    def execute_single_level_7(self, mainterm): # MULTIPLE KEY STRUCTURES IN THE SAME MAINTERM
        print("--------execute_single_level_7--------")
        """
        Args
            mainterm -> Mainterm in focus.
        Returns
            None
        """
        # Only works when 'code' and 'codes' are available as subterms in a mainterm's 'term' list
        final_codes = []
        partial_codes = []
        for term in mainterm["term"]:
            try:
                code = term["code"]
                final_codes.append(code)
            except KeyError:
                codes = term["codes"]
                partial_codes.append(codes)
        print(f"Choices for Final codes: {final_codes}")
        print(f"Choices for un-finished codes: {partial_codes}")

In [5]:
class ParentExecute(Parser):
    def parent_execute(self, mainterm):
        """
        Args
            mainterm -> Mainterm in focus.
        Returns
            None
        """
        if {"title", "use"} == mainterm.keys():
            self.execute_group_1(mainterm)
        elif {"title", "see"} == mainterm.keys():
            self.execute_group_2(mainterm)
        elif {"title", "term"} == mainterm.keys():
            self.execute_group_3(mainterm)
        elif {"title", "tab"} == mainterm.keys():
            self.execute_group_5(mainterm)
        elif {"title", "codes"} == mainterm.keys():
            self.execute_group_6(mainterm)
        elif {"title", "code"} == mainterm.keys():
            self.execute_group_4(mainterm)
        elif {"title", "code", "term"} == mainterm.keys():
            self.execute_group_7(mainterm)
        elif {"title", "code", "_level"} == mainterm.keys():
            self.execute_group_4(mainterm)
        elif {"see", "_level"} == mainterm.keys():
            self.execute_group_2(mainterm)
        elif {"use", "_level"} == mainterm.keys():
            self.execute_group_1(mainterm)
        elif {"title", "see", "_level"} == mainterm.keys():
            self.execute_group_2(mainterm)
        else:
            print("################## NO MATCHES FOUND. SEE PARENT_EXEUTE() ##################")
            pprint(mainterm)
    
    def execute_group_1(self, mainterm):
        print("--------execute_group_1--------")
        """
        Args
            mainterm -> Mainterm in focus.
        Returns
            None
        """
        if isinstance(mainterm["use"], str): # If "use" value has no children
            self.execute_group_1_1(mainterm)
        elif {"tab", "__text"} == mainterm["use"].keys():
            self.execute_group_1_2(mainterm)
    
    def execute_group_1_1(self, mainterm):
        print("--------execute_group_1_1--------")
        """
        Args
            mainterm -> Mainterm in focus.
        Returns
            None
        """
        print(f"Use the code associated with term '{mainterm['use']}' in the PCS Table.")

    def execute_group_1_2(self, mainterm):
        print("--------execute_group_1_2--------")
        """
        Args
            mainterm -> Mainterm in focus.
        Returns
            None
        """
        table = mainterm["use"]["tab"]
        text = mainterm["use"]["__text"]
        print(f"Go to table {table}, located at sections {text}")
    
    def execute_group_2(self, mainterm):
        print("--------execute_group_2--------")
        """
        Args
            mainterm -> Mainterm in focus.
        Returns
            None
        """
        if isinstance(mainterm["see"], str): # If "see" value has no children
            self.execute_group_2_1(mainterm)
        elif {"codes", "__text"} == mainterm["see"].keys():
            self.execute_group_2_2(mainterm)
        elif {"tab", "__text"} == mainterm["see"].keys():
            self.execute_group_2_3(mainterm)
    
    def execute_group_2_1(self, mainterm):
        print("--------execute_group_2_1--------")
        """
        Args
            mainterm -> Mainterm in focus.
        Returns
            None
        """
        new_term = mainterm["see"]
        print(f"Redirected to term '{new_term}'.")
        for new_mainterm_1 in mainterms: # mainterms -> list of mainterms, global variable must be established
            if new_mainterm_1["title"] == new_term:
                print(f"FOUND {new_term} ON FIRST ATTEMPT")
                self.execute_tree(new_mainterm_1)
                break
            else:
                split_term = new_term.split(", ")
                if len(split_term) == 2:
                    self.execute_group_2_1_1(split_term)
                elif len(split_term) >= 3:
                    print("ERROR 1: Found 'see' tag that wasn't identified on re-query. Need to write function for specific use case. See execute_group_2_1().")
                    pprint(mainterm)
    
    def execute_group_2_1_1(self, split_term):
        print("--------execute_group_2_1_1--------")
        new_search_parent, new_search_child = split_term
        print(f"LOOKING FOR NEW PARENT TERM: {new_search_parent}")
        for new_mainterm_2 in mainterms:
            if new_mainterm_2["title"] == new_search_parent:
                for subterm in new_mainterm_2["term"]:
                    if subterm["title"] == new_search_child:
                        print(f"FOUND CHILD TERM: {new_search_child}")
                        self.execute_tree(subterm)
                        break
    
    def execute_group_2_2(self, mainterm):
        print("--------execute_group_2_2--------")
        code = mainterm["see"]["codes"]
        text = mainterm["see"]["__text"]
        print(f"""Go to table '{code[:3]}', located at '{text}'.
Position 4 and greater code(s): '{code[3:]}'
Each code corresponds to it's respective position number.""")
        
    def execute_group_2_3(self, mainterm):
        print("--------execute_group_2_3--------")
        table = mainterm["see"]["tab"]
        text = mainterm["see"]["__text"]
        print(f"Go to table '{table}', located at sections '{text}'.")
    
    def execute_group_3(self, mainterm):
        print("--------execute_group_3--------")
        # If user is met with a "term" tag that means there's levels of code
        if {'title', 'codes', '_level'} == mainterm.keys():
            self.execute_group_6(mainterm)  
        elif {'title', 'code', '_level'} == mainterm.keys():
            self.execute_group_4_1(mainterm)
        elif {'see', '_level'} == mainterm.keys():
            self.execute_group_2(mainterm)
        elif {'use', '_level'} == mainterm.keys():
            self.execute_group_1(mainterm)
        elif {'title', 'see', '_level'} == mainterm.keys():
            self.execute_group_2(mainterm)
        elif {'code', '_level'} == mainterm.keys():
            self.execute_group_4(mainterm)
        elif {'title', 'tab', '_level'} == mainterm.keys():
            self.execute_group_5(mainterm)
        elif {"_level"} == mainterm.keys():
            print("No term given. Search your term again and choose a different option.")
            pprint(mainterm)
        else:
            print("################## NO MATCHES FOUND. SEE EXECUTE_GROUP_3() ##################")
            pprint(mainterm)
            
    def execute_group_4(self, mainterm):
        print("--------execute_group_4--------")
        self.execute_group_4_1(mainterm)
    
    def execute_group_4_1(self, mainterm):
        print("--------execute_group_4_1--------")
        print(f"Use final code: {mainterm['code']} with description '{mainterm['title']}'.")
        
    def execute_group_5(self, mainterm):
        print("--------execute_group_5--------")
        self.execute_group_5_1(mainterm)
    
    def execute_group_5_1(self, mainterm):
        print("--------execute_group_5_1--------")
        table = mainterm["tab"]
        print(f"Go to table: {table}. No additional guidance given.")

    def execute_group_6(self, mainterm):
        print("--------execute_group_6--------")
        self.execute_group_6_1(mainterm)
    
    def execute_group_6_1(self, mainterm):
        print("--------execute_group_6_1--------")
        code = mainterm["codes"]
        print(f"""Go to table: {code[:3]}
Position 4 and greater code(s): '{code[3:]}'
Each code corresponds to it's respective position number.""")

    def execute_group_7(self, mainterm):
        print("--------execute_group_7--------")
        self.execute_group_7_1(mainterm)
        
    def execute_group_7_1(self, mainterm):
        print("--------execute_group_7_1--------")
        code = mainterm["code"]
        level_1_terms = [term["title"]
                        for term in mainterm["term"]
                        if term["_level"] == "1"]
        user_input = input(f"""Use code: {code}. If this case involves any of the following terms, {level_1_terms}, press 'y' now. If not, press 'n' to exit.""")
        if user_input == "y":
            self.execute_tree(mainterm)

## Testing Block

In [8]:
parser = Parser()
# single = SingleLevel()
# parent = ParentExecute()

In [9]:
# Creating generator object of all mainterms
letters = index["ICD10PCS.index"]["letter"]
mainterms = (mainterms for letter in letters for mainterms in letter["mainTerm"])
# mainterms = [mainterms for letter in letters for mainterms in letter["mainTerm"]]
# Searching for user's lookup term
user_mainterm = "DownStream(R) System" # Only for testing purposes, user will pick
for mainterm in mainterms:
    if mainterm["title"] == user_mainterm:
        parser.execute_tree(mainterm, True)

--------EXECUTE SINGLE LEVEL--------
--------execute_single_level_5--------
You have 2 choices for final codes related to term DownStream(R) System Choose the code that works best for this medical case.
['5A0512C', '5A0522C']
